In [7]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = f"/content/drive/MyDrive/.kaggle"
!kaggle competitions download -c bias-correction-gs-ma-p
!mkdir kaggle_dataset
!unzip -q /content/bias-correction-gs-ma-p.zip -d /content/kaggle_dataset
!rm /content/bias-correction-gs-ma-p.zip

 58% 121M/210M [00:00<00:00, 1.26GB/s]
100% 210M/210M [00:00<00:00, 793MB/s] 
mkdir: cannot create directory ‘kaggle_dataset’: File exists


In [8]:
!pip install torch_geometric

In [9]:
import pandas as pd
import torch
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader
import numpy as np
from sklearn.metrics.pairwise import haversine_distances,euclidean_distances
from sklearn.preprocessing import MinMaxScaler

# Prepare Graph Data
Making Human friendly Pipeline

## Prepare edge indices

In [10]:
coor_df = pd.read_csv('/content/kaggle_dataset/Coor_HII_495sta.csv')

gsmap_df = pd.read_csv('/content/kaggle_dataset/Train/GSMap_now_station_2015-2020.csv')
hll_df = pd.read_csv('/content/kaggle_dataset/Train/HII_station_2015-2020.csv')
humid_df = pd.read_csv('/content/kaggle_dataset/Train/Humidity_2015-2020.csv')
pressure_df = pd.read_csv('/content/kaggle_dataset/Train/Pressure_2015-2020.csv')
temp_df = pd.read_csv('/content/kaggle_dataset/Train/Temperature_2015-2020.csv')

In [11]:
def scale_from_column_4(df):
    # Extract columns to scale (from index 4 onward)
    columns_to_scale = df.columns[4:]
    # Initialize the scaler
    scaler = MinMaxScaler()
    # Scale the selected columns
    scaled_values = scaler.fit_transform(df[columns_to_scale])
    # Replace the original columns with scaled values
    df[columns_to_scale] = scaled_values
    return df

# Apply scaling to each DataFrame
coor_rad = coor_df[['lat', 'long']]* np.pi / 180 #Convert to radians
gsmap_df = scale_from_column_4(gsmap_df)
humid_df = scale_from_column_4(humid_df)
pressure_df = scale_from_column_4(pressure_df)
temp_df = scale_from_column_4(temp_df)

In [12]:
scaler_hll = MinMaxScaler()
columns_to_scale = hll_df.columns[4:]
scaled_values = scaler_hll.fit_transform(hll_df[columns_to_scale])
hll_df[columns_to_scale] = scaled_values
hll_df

,Year,Month,Day,Hour,ACRU,BAKI,BARI,BBHN,BBON,BBUA,...,WTSG,YGHM,YOM001,YOM003,YOM005,YOM006,YOM007,YOM008,YOM009,YOM010
0,2015,1,1,0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2015,1,1,1,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2015,1,1,2,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2015,1,1,3,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2015,1,1,4,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52603,2020,12,31,19,0.0,0.0,0.0,0.010033,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52604,2020,12,31,20,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52605,2020,12,31,21,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52606,2020,12,31,22,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
distance_threshold = 100  # Define a distance threshold for km

distance_matrix = haversine_distances(coor_rad[['lat', 'long']]) #compute distance matrix
distance_matrix = distance_matrix * 6371

i, j = np.where((distance_matrix < distance_threshold) & (distance_matrix != 0))
edge_index = np.column_stack((i, j))
edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

pd.DataFrame(distance_matrix, index=coor_df['Station'], columns=coor_df['Station'])

Station,ACRU,BAKI,BARI,BBHN,BBON,BBUA,BBWN,BCNG,BDAR,BDCP,...,WTSG,YGHM,YOM001,YOM003,YOM005,YOM006,YOM007,YOM008,YOM009,YOM010
Station,,,,,,,,,,,,,,,,,,,,,
ACRU,0.000000,240.856550,554.417048,956.400917,641.221515,189.243770,1003.817715,228.796404,265.589814,430.957680,...,496.333629,648.685166,594.438522,548.489601,554.280680,533.181749,515.140543,488.011232,470.536495,546.592946
BAKI,240.856550,0.000000,503.761695,1039.943117,435.988940,64.613833,1091.075310,29.571721,351.260040,582.356453,...,446.708987,416.272261,373.379966,351.311650,389.200479,381.641804,376.337181,360.819608,366.282576,354.843322
BARI,554.417048,503.761695,0.000000,624.419507,415.874464,539.942431,676.319467,477.432629,324.913119,446.063090,...,59.287135,544.660256,459.709873,361.609304,272.521787,238.513809,211.017465,194.119598,158.592636,345.116545
BBHN,956.400917,1039.943117,624.419507,0.000000,1036.632157,1048.059935,52.342747,1010.373747,704.895795,547.146751,...,655.482256,1169.052404,1084.119583,985.939287,896.441377,862.916238,835.174991,815.620609,774.721800,969.496872
BBON,641.221515,435.988940,415.874464,1036.632157,0.000000,500.186528,1087.939567,430.950166,568.556988,784.522470,...,399.993509,148.930704,89.848526,94.228826,147.140046,184.114110,215.722216,245.112295,290.788837,100.807429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YOM006,533.181749,381.641804,238.513809,862.916238,184.114110,437.833350,914.780149,364.761658,404.869489,607.244615,...,216.572365,306.146476,221.323072,123.711276,37.131397,0.000000,31.848916,63.685355,108.163919,107.017446
YOM007,515.140543,376.337181,211.017465,835.174991,215.722216,429.912139,887.158054,357.468804,376.262285,575.928196,...,185.892098,334.516208,249.160125,150.814606,68.958068,31.848916,0.000000,33.136708,76.444958,134.508010
YOM008,488.011232,360.819608,194.119598,815.620609,245.112295,411.640032,867.771276,340.184014,343.221036,543.583577,...,161.830352,357.241429,271.565035,173.467327,100.053071,63.685355,33.136708,0.000000,45.737807,157.985875


## Prepare Features

In [14]:
#Prepare 1 Graph per Day
x = torch.tensor(gsmap_df.iloc[0,4:])
y = torch.tensor(hll_df.iloc[0,4:])
dataset = Data(x=x,
            y=y,
            edge_index=edge_index)
dataset

<ipython-input-14-2186647695>:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = torch.tensor(gsmap_df.iloc[0,4:])
<ipython-input-14-2186647695>:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = torch.tensor(hll_df.iloc[0,4:])


Data(x=[495], edge_index=[2, 13404], y=[495])

In [15]:
print(dataset.num_nodes,
      dataset.num_edges,
      dataset.num_node_features,
      dataset.has_isolated_nodes(),
      dataset.has_self_loops(),
      dataset.is_directed())

495 13404 1 False False False


## Prepare Loader

In [16]:
from torch_geometric.data import Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.data import InMemoryDataset
from sklearn.model_selection import train_test_split

In [17]:
# --- Define Custom Dataset for Node Regression ---
class DailyGraphDataset(Dataset):
    def __init__(self, hii_df, gsmap_df, humid_df, pressure_df, temp_df, edge_index):
        super(DailyGraphDataset, self).__init__()
        self.hii_df = hii_df  # Targets (actual rainfall)
        self.gsmap_df = gsmap_df  # Feature: GSMAP estimates
        self.humid_df = humid_df  # Feature: Humidity
        self.pressure_df = pressure_df  # Feature: Pressure
        self.temp_df = temp_df  # Feature: Temperature
        self.edge_index = edge_index  # Consistent edge indices for all graphs
        self.num_feature = 4

    def __len__(self):
        return len(self.gsmap_df)

    def __getitem__(self, idx):
        gsmap_values = self.gsmap_df.iloc[idx, 4:].values  # Skip first 4 columns (metadata)
        humid_values = self.humid_df.iloc[idx, 4:].values
        pressure_values = self.pressure_df.iloc[idx, 4:].values
        temp_values = self.temp_df.iloc[idx, 4:].values

        x = torch.tensor(
            np.column_stack((gsmap_values, humid_values, pressure_values, temp_values)),
            dtype=torch.float
        )  # Shape: [num_stations, num_features]

        # Node targets: Actual rainfall (HII) for regression
        y = torch.tensor(self.hii_df.iloc[idx, 4:].values, dtype=torch.float)  # Shape: [num_stations]

        # Create a graph for this timestamp
        return Data(x=x, y=y, edge_index=self.edge_index)


In [18]:
# --- Split Dataset into Train and Validation ---
indices = list(range(len(gsmap_df)))
train_indices, val_indices = train_test_split(indices, test_size=0.2, random_state=42)

train_ds = DailyGraphDataset(
    hll_df.iloc[train_indices], gsmap_df.iloc[train_indices], humid_df.iloc[train_indices],
    pressure_df.iloc[train_indices], temp_df.iloc[train_indices], edge_index
)
val_ds = DailyGraphDataset(
    hll_df.iloc[val_indices], gsmap_df.iloc[val_indices], humid_df.iloc[val_indices],
    pressure_df.iloc[val_indices], temp_df.iloc[val_indices], edge_index
)

BATCH_SIZE = 256
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

In [19]:
# --- Verify the First Batch ---
for batch in train_dl:
    print("First Batch Details:")
    print(f"Batch Shape: {batch}")
    print(f"Node Features Shape: {batch.x.shape}")  # Expected: [batch_size * num_nodes, num_features]
    print(f"Edge Index Shape: {batch.edge_index.shape}")  # Expected: [2, num_edges * batch_size]
    print(f"Target Shape: {batch.y.shape}")  # Expected: [batch_size * num_nodes]
    print(f"Batch Vector: {batch.batch.shape}")  # Indicates which nodes belong to which graph
    print(f"Number of Graphs in Batch: {batch.num_graphs}")
    break

# --- Example: Inspect a Single Graph ---
single_graph = train_ds[0]
print("\nSingle Graph Details:")
print(f"Number of Nodes: {single_graph.num_nodes}")  # Should be 495
print(f"Number of Edges: {single_graph.num_edges}")  # Should be 13404
print(f"Node Features Shape: {single_graph.x.shape}")  # Should be [495, 4]
print(f"Target Shape: {single_graph.y.shape}")  # Should be [495]
print(f"Has Isolated Nodes: {single_graph.has_isolated_nodes()}")
print(f"Has Self Loops: {single_graph.has_self_loops()}")
print(f"Is Directed: {single_graph.is_directed()}")

First Batch Details:
Batch Shape: DataBatch(x=[126720, 4], edge_index=[2, 3431424], y=[126720], batch=[126720], ptr=[257])
Node Features Shape: torch.Size([126720, 4])
Edge Index Shape: torch.Size([2, 3431424])
Target Shape: torch.Size([126720])
Batch Vector: torch.Size([126720])
Number of Graphs in Batch: 256

Single Graph Details:
Number of Nodes: 495
Number of Edges: 13404
Node Features Shape: torch.Size([495, 4])
Target Shape: torch.Size([495])
Has Isolated Nodes: False
Has Self Loops: False
Is Directed: False


# Model

In [20]:
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
from tqdm import tqdm
from torch import nn

In [21]:
# --- Define GNN Model ---
class GNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.fc(x)
        return x.squeeze(-1)  # [num_nodes], remove last dimension for regression

In [119]:
# --- Training Loop ---
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    total_mae = 0
    num_samples = 0

    for batch in dataloader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch)  # [batch_size * num_nodes]
        loss = criterion(out, batch.y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * batch.num_nodes
        total_mae += torch.abs(out - batch.y).sum().item()
        num_samples += batch.num_nodes

    avg_loss = total_loss / num_samples
    avg_mae = total_mae / num_samples
    return avg_loss, avg_mae

def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    total_mae = 0
    num_samples = 0

    with torch.no_grad():
        for batch in dataloader:
            batch = batch.to(device)
            out = model(batch)
            loss = criterion(out, batch.y)

            total_loss += loss.item() * batch.num_nodes
            total_mae += torch.abs(out - batch.y).sum().item()
            num_samples += batch.num_nodes

    avg_loss = total_loss / num_samples
    avg_mae = total_mae / num_samples
    return avg_loss, avg_mae

In [120]:
# --- Initialize Model, Loss, and Optimizer ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNNModel(input_dim=4, hidden_dim=32, output_dim=1).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

In [121]:
num_epochs = 10
print(f"Starting training... CUDA : {torch.cuda.is_available()}")
for epoch in tqdm(range(num_epochs),colour="green"):
    train_loss, train_mae = train_epoch(model, train_dl, criterion, optimizer, device)
    val_loss, val_mae = evaluate(model, val_dl, criterion, device)
    print(f"Epoch {epoch+1}/{num_epochs} | "
          f"Train Loss: {train_loss:.4f} | Train MAE: {train_mae:.4f} | "
          f"Val Loss: {val_loss:.4f} | Val MAE: {val_mae:.4f}")

Starting training... CUDA : True


  0%|          | 0/10 [00:10<?, ?it/s]


KeyboardInterrupt: 

# Inference

In [30]:
# --- Define Custom Dataset for Node Regression ---
class TEST_DailyGraphDataset(Dataset):
    def __init__(self, gsmap_df, humid_df, pressure_df, temp_df, edge_index):
        super(TEST_DailyGraphDataset, self).__init__()
        self.gsmap_df = gsmap_df  # Feature: GSMAP estimates
        self.humid_df = humid_df  # Feature: Humidity
        self.pressure_df = pressure_df  # Feature: Pressure
        self.temp_df = temp_df  # Feature: Temperature
        self.edge_index = edge_index  # Consistent edge indices for all graphs
        self.num_feature = 4

    def __len__(self):
        return len(self.gsmap_df)

    def __getitem__(self, idx):
        gsmap_values = self.gsmap_df.iloc[idx, 4:].values  # Skip first 4 columns (metadata)
        humid_values = self.humid_df.iloc[idx, 4:].values
        pressure_values = self.pressure_df.iloc[idx, 4:].values
        temp_values = self.temp_df.iloc[idx, 4:].values

        x = torch.tensor(
            np.column_stack((gsmap_values, humid_values, pressure_values, temp_values)),
            dtype=torch.float
        )  # Shape: [num_stations, num_features]

        # Create a graph for this timestamp
        return Data(x=x, edge_index=self.edge_index)


In [55]:
TEST_gsmap_df = pd.read_csv('/content/kaggle_dataset/Test/GSMap_now_station_2021-2024.csv')
TEST_humid_df = pd.read_csv('/content/kaggle_dataset/Test/Humidity_2021-2024.csv')
TEST_pressure_df = pd.read_csv('/content/kaggle_dataset/Test/Pressure_2021-2024.csv')
TEST_temp_df = pd.read_csv('/content/kaggle_dataset/Test/Temperature_2021-2024.csv')
submission = pd.read_csv('/content/kaggle_dataset/sample_submission.csv')

In [79]:
BATCH_SIZE = 256
test_ds = TEST_DailyGraphDataset(
    TEST_gsmap_df, TEST_humid_df,TEST_pressure_df, TEST_temp_df, edge_index
)
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

## Predict each datapoint

In [60]:
predictions = []  # Changed variable name to plural for clarity
# Assuming device is defined (e.g., device = torch.device("cuda" if torch.cuda.is_available() else "cpu"))
print(device)
model.eval()
with torch.no_grad():
    for data in tqdm(test_ds,colour = "green"):
        data = data.to(device)  # Assumes batch is a tensor or a tuple/dict of tensors
        out = model(data)  # Forward pass through the model
        predictions.extend(out.cpu().numpy())  # Convert to numpy and extend predictions

cuda


100%|██████████| 35064/35064 [01:57<00:00, 297.80it/s]


In [71]:
reshaped_prediction = np.array(predictions).reshape(-1,495)
reshaped_prediction.shape

(35064, 495)

In [76]:
df = pd.DataFrame(reshaped_prediction)
df

,0,1,2,3,4,5,6,7,8,9,...,485,486,487,488,489,490,491,492,493,494
0,44.121998,39.456047,35.502331,41.230923,42.554440,38.827175,38.203148,39.838024,38.592339,32.720318,...,35.599545,36.896763,41.206879,39.503513,41.601933,40.231998,39.050495,40.593121,41.092350,39.005569
1,44.212337,39.594494,35.654736,41.255833,42.637257,38.953087,38.216015,39.984707,38.745861,32.722256,...,35.761623,36.939285,41.324951,39.709484,41.808636,40.454247,39.275436,40.818913,41.307369,39.214668
2,44.315022,39.723660,35.757473,41.217098,42.654919,39.080299,38.190990,40.118153,38.892471,32.748081,...,35.926525,36.954445,41.375835,39.810520,41.968674,40.676128,39.529125,41.092808,41.589474,39.324150
3,44.398350,39.798595,35.761826,41.184769,42.742268,39.147507,38.153278,40.193954,39.028210,32.803471,...,35.941818,37.002911,41.487576,39.985172,42.121601,40.822273,39.665783,41.215416,41.680939,39.489532
4,44.491173,39.858463,35.834568,41.154041,42.836117,39.211243,38.170109,40.266563,39.124805,32.846596,...,36.052620,37.090149,41.602844,40.086670,42.254719,40.971973,39.820053,41.380928,41.848804,39.593510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,43.086952,37.985958,35.069069,40.556828,42.200596,37.392437,37.566612,38.504459,38.238346,32.613220,...,35.434795,36.821896,41.140125,39.366199,41.432869,40.204910,39.147827,40.711140,41.120308,38.853809
35060,43.336323,38.259480,35.487114,40.981464,42.652920,37.654308,37.931759,38.772701,38.666252,32.796757,...,35.839947,37.165455,41.570446,39.836060,41.940277,40.692184,39.605438,41.183792,41.597027,39.322300
35061,43.543827,38.596272,35.738251,41.173130,42.916531,37.982082,38.134823,39.090382,38.912571,33.056328,...,36.093685,37.388279,41.828983,40.116928,42.253387,41.001484,39.899609,41.484699,41.901600,39.603573
35062,43.692890,38.817051,35.897243,41.413128,43.145359,38.242336,38.347214,39.292439,39.023235,33.309223,...,36.252480,37.540180,42.033184,40.353561,42.473721,41.199818,40.086777,41.675331,42.095520,39.834053


## Predict in Batch

In [111]:
predictions = np.array([])  # Changed variable name to plural for clarity
# Assuming device is defined (e.g., device = torch.device("cuda" if torch.cuda.is_available() else "cpu"))
print(device)
model.eval()
with torch.no_grad():
    for batch in tqdm(test_dl,colour = "green"):
        batch = batch.to(device)  # Assumes batch is a tensor or a tuple/dict of tensors
        out = model(batch)  # Forward pass through the model
        predictions = np.append(predictions,out.cpu().numpy())


cuda


100%|██████████| 137/137 [01:00<00:00,  2.26it/s]


In [115]:
reshaped_prediction = np.array(predictions).reshape(-1,495)
reshaped_prediction.shape

(35064, 495)

In [117]:
predictions

array([44.12201309, 39.45605087, 35.50233841, ..., 41.80929565,
       42.25159073, 40.0100708 ])

In [116]:
df = pd.DataFrame(reshaped_prediction)
df

,0,1,2,3,4,5,6,7,8,9,...,485,486,487,488,489,490,491,492,493,494
0,44.122013,39.456051,35.502338,41.230907,42.554440,38.827175,38.203148,39.838028,38.592346,32.720322,...,35.599545,36.896759,41.206882,39.503498,41.601944,40.231998,39.050495,40.593124,41.092346,39.005569
1,44.212326,39.594494,35.654724,41.255829,42.637260,38.953083,38.216003,39.984715,38.745857,32.722248,...,35.761623,36.939289,41.324951,39.709480,41.808632,40.454243,39.275436,40.818909,41.307373,39.214664
2,44.315033,39.723656,35.757469,41.217094,42.654911,39.080299,38.191002,40.118153,38.892475,32.748081,...,35.926521,36.954441,41.375832,39.810516,41.968662,40.676136,39.529118,41.092823,41.589478,39.324150
3,44.398350,39.798599,35.761822,41.184753,42.742283,39.147507,38.153286,40.193943,39.028206,32.803471,...,35.941814,37.002918,41.487568,39.985176,42.121597,40.822285,39.665775,41.215427,41.680943,39.489521
4,44.491184,39.858467,35.834572,41.154034,42.836121,39.211246,38.170116,40.266563,39.124805,32.846600,...,36.052620,37.090153,41.602852,40.086674,42.254726,40.971973,39.820057,41.380928,41.848804,39.593513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,43.086964,37.985958,35.069069,40.556828,42.200600,37.392429,37.566605,38.504456,38.238350,32.613220,...,35.434803,36.821896,41.140133,39.366192,41.432850,40.204903,39.147831,40.711132,41.120312,38.853813
35060,43.336319,38.259480,35.487103,40.981461,42.652912,37.654308,37.931763,38.772697,38.666248,32.796764,...,35.839931,37.165455,41.570446,39.836060,41.940277,40.692196,39.605438,41.183800,41.597031,39.322308
35061,43.543823,38.596275,35.738258,41.173130,42.916534,37.982082,38.134827,39.090385,38.912575,33.056332,...,36.093693,37.388279,41.828979,40.116928,42.253387,41.001495,39.899612,41.484699,41.901600,39.603584
35062,43.692898,38.817051,35.897243,41.413128,43.145359,38.242344,38.347218,39.292439,39.023235,33.309227,...,36.252480,37.540169,42.033184,40.353565,42.473717,41.199802,40.086773,41.675327,42.095528,39.834053
